In [1]:
import time
import pickle 
import random
import joblib

import pandas as pd
import numpy as np
from numpy.linalg import norm

import scipy.sparse as sparse
from sklearn.metrics.pairwise import cosine_similarity

import implicit
from implicit.nearest_neighbours import bm25_weight
from implicit import evaluation

import ml_metrics

random_seed = 0
np.random.seed(random_seed)

/home/valkea/Dev/OpenClassrooms/Projets_AI/P9/venvP9/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
scores_df = pd.DataFrame([], columns=['model_name', 'mean_cosine_similarity', 'precision@k', 'map@k', 'ndcg@k', 'training_time'])
scores_df.set_index('model_name', inplace=True)
scores_df.head()

,mean_cosine_similarity,precision@k,map@k,ndcg@k,training_time
model_name,,,,,


### Chargeons les jeux de données `training` et `validation`

In [3]:
data_train = pd.read_csv('data/data_train.csv')
display(data_train.head(3), data_train.shape)

,user_id,article_id,category_id,score
0,59,234853,375,-0.328045
1,79,159359,281,-0.327253
2,154,96663,209,-0.363316


(1577295, 4)

In [4]:
data_valid = pd.read_csv('data/data_valid.csv')
display(data_valid.head(3), data_valid.shape)

,user_id,article_id,category_id,score
0,279777,96210,209,-0.387729
1,29634,284773,412,-0.209496
2,55,162605,281,0.213075


(241105, 4)

### Préparons une sparse matrix pour entrainer nos algorithmes de collaborative filtering

In [5]:
# --- Train ---
data_train['user_cat_code'] = data_train['user_id'].astype('category').cat.codes
data_train['article_cat_code'] = data_train['article_id'].astype('category').cat.codes

# train_sparse_item_user = sparse.csr_matrix((data_train['score'].astype(float), (data_train['article_cat_code'], data_train['user_cat_code'])))
train_sparse_user_item = sparse.csr_matrix((data_train['score'].astype(float), (data_train['user_cat_code'], data_train['article_cat_code'])))
display(train_sparse_user_item.shape)

# --- Validation ---
data_valid['user_cat_code'] = data_valid['user_id'].astype('category').cat.codes
data_valid['article_cat_code'] = data_valid['article_id'].astype('category').cat.codes

# valid_sparse_item_user = sparse.csr_matrix((data_valid['score'].astype(float), (data_valid['article_cat_code'], data_valid['user_cat_code'])))
valid_sparse_user_item = sparse.csr_matrix((data_valid['score'].astype(float), (data_valid['user_cat_code'], data_valid['article_cat_code'])))

display(valid_sparse_user_item.shape)

(297141, 28002)

(84041, 7576)

#### Et préparons une fonction lookup pour retrouver les article_id à partir des article_cat_code

In [6]:
article_lookup = pd.DataFrame(data_train.groupby('article_cat_code')['article_id'].apply(lambda x: list(x)[0])).to_dict()

def lookup_articles(x):
    try:
        return article_lookup['article_id'][x]
    except Exception:
        return -1

### Chargons les embeddings

In [7]:
file = open('data/news-portal-user-interactions-by-globocom/articles_embeddings.pickle',"rb")
article_embedding = pickle.load(file)

In [8]:
display(article_embedding[:5], article_embedding.shape)

array([[-0.16118301, -0.95723313, -0.13794445, ..., -0.231686  ,
         0.5974159 ,  0.40962312],
       [-0.52321565, -0.974058  ,  0.73860806, ...,  0.18282819,
         0.39708954, -0.83436364],
       [-0.61961854, -0.9729604 , -0.20736018, ..., -0.44758022,
         0.8059317 , -0.28528407],
       [-0.7408434 , -0.97574896,  0.39169782, ..., -0.5378381 ,
         0.24354108, -0.8853287 ],
       [-0.2790515 , -0.97231525,  0.68537366, ..., -0.42406067,
         0.18548405, -0.5802922 ]], dtype=float32)

(364047, 250)

## Calculons la cosine_similarity entre le data_train et le data_valid *(pour chaque utilisateur, puis la moyenne)*

In [9]:
def get_train_valid_similarity( data_t, data_v ):

    select = pd.DataFrame()
    
    # --- select the validation user_ids
    user_idx = data_v.user_id.unique()
    
    # --- for each user, get the ids of the articles he/she has read in the TRAINING set et le VALIDATION set
    select['article_ids_train'] = data_t[data_t.user_id.isin(user_idx)].groupby('user_id')['article_id'].apply(list)
    select['article_ids_valid'] = data_v[data_v.user_id.isin(user_idx)].groupby('user_id')['article_id'].apply(list)
    
    # --- remove user_id with only 1 article either the TRAINING or the VALIDATION set
    select = select[select['article_ids_train'].map(len) > 1]
    select = select[select['article_ids_valid'].map(len) > 1]
    select.dropna(axis=0, inplace=True)
    
    # --- for each user, compute the mean embedding vectors of the articles he/she has read
    select['mean_embedding_train'] = select['article_ids_train'].apply(lambda x : article_embedding[x].mean(axis=0))
    select['mean_embedding_valid'] = select['article_ids_valid'].apply(lambda x : article_embedding[x].mean(axis=0))

    # --- for each user, compute the cosine similarity between the user's mean_embeddings in the TRAINING set and in the VALIDATION set
    select['cosine'] = select.apply(lambda x: cosine_similarity(x['mean_embedding_train'].reshape(1, -1), x['mean_embedding_valid'].reshape(1, -1))[0][0], axis=1)

    # --- Compute & return overall mean cosine similarity
    return select.cosine.mean(), select

cosine_train_valid, cosine_train_valid_df = get_train_valid_similarity(data_train, data_valid)

print(f"\nmean_cosine_similarity (de 1 à -1) entre 'viewed_in_train' & 'viewed_in_valid' = {cosine_train_valid:.2f}")


mean_cosine_similarity (de 1 à -1) entre 'viewed_in_train' & 'viewed_in_valid' = 0.64


In [10]:
cosine_train_valid_df.head(3)

,article_ids_train,article_ids_valid,mean_embedding_train,mean_embedding_valid,cosine
user_id,,,,,
5,"[168868, 156543, 284664, 156672, 59929, 160974...","[225010, 69353, 161872, 205845, 57748, 157815,...","[-0.17539436, -0.9664814, -0.0456632, -0.30515...","[-0.024590481, -0.9645934, -0.06022447, -0.023...",0.917003
7,"[235840, 156624, 123757, 64409, 336221, 183176]","[199474, 87223, 352979, 284470, 36162, 156279]","[-0.32671222, -0.97076577, -0.31606814, -0.361...","[-0.11400774, -0.96468645, -0.32728586, -0.201...",0.739961
8,"[332114, 284847, 272660, 273464, 313504, 23513...","[331116, 96141, 234481]","[-0.47689843, -0.9678452, -0.2180024, -0.47295...","[-0.32962552, -0.97033435, 0.20547153, -0.0870...",0.741852


## Calculons ce même score de cosine similarity si l'on donne des articles au hasard aux utilisateurs *(baseline)*

In [11]:
def get_mean_cosine_similarity(data_ref, reco_ref, reco_model, reco_size = 5, top_users=None):

    # --- for each user, get the ids of the articles he/she has read
    select = data_ref.groupby('user_id')['article_id'].apply(list).reset_index(name='article_ids')
    
    # --- remove user if he/she has only one article in the data_ref (probably data_valid)
    select = select[select.article_ids.map(len) > 1]
    
    # --- crop selection if specified
    if top_users is not None:
        select = select[:top_users]
        
    # print("users_ids:", select.user_id)

    # --- for each user, compute the mean embedding vectors of the articles he/she has read
    select['read_mean_embedding'] = select.apply(lambda x : article_embedding[np.array(x[1])].mean(axis=0), axis=1)

    # --- for each user, make recommendations
    select['reco_article_ids'] = reco_model(select.index, select.read_mean_embedding, reco_ref, reco_size)

    # --- for each user, compute the mean embedding vectors of the recommended articles
    select['reco_mean_embedding'] = select['reco_article_ids'].apply(lambda x : article_embedding[np.array(x)].mean(axis=0))

    # --- for each user, compute the cosine similarity between the read_mean_embedding and the pred_mean_embedding
    select['cosine'] = select.apply(lambda x: cosine_similarity(x['read_mean_embedding'].reshape(1, -1), x['reco_mean_embedding'].reshape(1, -1))[0][0], axis=1)

    # --- reset the index column
    select = select.set_index('user_id')

    # --- Compute & return overall mean cosine similarity
    return select.cosine.mean(), select

In [12]:
def reco_random(user_ids, mean_embeddings, reco_ref, reco_size):
    # --- ensure to get the same random matrix for reproductibility
    np.random.seed(random_seed)
    
    # --- set 5 random article_id for each users
    return list(np.random.randint(len(article_embedding), size=(len(user_ids), reco_size)))

#### Pour tout le jeu de données

In [13]:
MCS, MCS_df = get_mean_cosine_similarity(data_valid, valid_sparse_user_item, reco_random, 5)

display(MCS_df.head(), MCS_df.shape)
print(f"\nmean_cosine_similarity: {MCS:.2f} (sachant que la cosine similarity va de 1 à -1)")

,article_ids,read_mean_embedding,reco_article_ids,reco_mean_embedding,cosine
user_id,,,,,
5,"[225010, 69353, 161872, 205845, 57748, 157815,...","[-0.024590481, -0.9645934, -0.06022447, -0.023...","[305711, 117952, 152315, 358083, 359783]","[-0.15132096, -0.956843, 0.125719, -0.27082413...",0.557499
7,"[199474, 87223, 352979, 284470, 36162, 156279]","[-0.11400774, -0.96468645, -0.32728586, -0.201...","[304137, 122579, 86293, 211543, 212038]","[-0.4073767, -0.959381, -0.1800201, -0.0457417...",0.672065
8,"[331116, 96141, 234481]","[-0.32962552, -0.97033435, 0.20547153, -0.0870...","[310744, 170584, 314764, 80186, 17089]","[-0.4528594, -0.96688926, 0.4099636, -0.472056...",0.585191
10,"[196588, 193449, 195689, 65991, 100931, 58556,...","[-0.23681411, -0.96241695, -0.08872973, -0.346...","[150055, 220760, 363345, 255653, 82457]","[-0.3074916, -0.96819973, 0.09544015, -0.04938...",0.642696
11,"[208582, 96877, 195177, 205824]","[-0.15431535, -0.9709704, 0.069619074, 0.13036...","[329843, 332752, 7877, 346110, 73135]","[0.16090424, -0.96745205, -0.013877422, -0.109...",0.418073


(46638, 5)


mean_cosine_similarity: 0.46 (sachant que la cosine similarity va de 1 à -1)


#### Pour les 1000 premiers utilisateurs *(car j'ai du mal à calculer le score sur l'ensemble du jeu de données avec le Content Based Filtering)*

In [14]:
MCS, MCS_df = get_mean_cosine_similarity(data_valid, valid_sparse_user_item, reco_random, 5, 1000)
print(f"\nmean_cosine_similarity: {MCS:.2f} (sachant que la cosine similarity va de 1 à -1)")

model_name = "Content Based Filtering - Random"
scores_df.at[model_name,'mean_cosine_similarity'] = MCS
scores_df.at[model_name,'training_time'] = 0


mean_cosine_similarity: 0.48 (sachant que la cosine similarity va de 1 à -1)


#### Vérifions la distribution des cosine_similarity calculés

In [15]:
MCS_df.describe().T

,count,mean,std,min,25%,50%,75%,max
cosine,1000.0,0.482212,0.13109,0.04751,0.394346,0.495873,0.57634,0.826292


# 1. Candidate generation

## 1.1 Collaborative Filtering

### Utilisons un système permettant de rééquilibrer les notes implicites *(pour éviter de donner trop d'importance aux articles qui ont un très gros ratio `temps de lecture` / `nombre de mots`)*

In [16]:
# train_sparse_item_user_bm25 = bm25_weight(sparse_item_user, K1=100, B=0.9).tocsr()
train_sparse_user_item_bm25 = bm25_weight(train_sparse_user_item, K1=100, B=0.9).tocsr() # Implicit veut des matrices [user x item]
valid_sparse_user_item_bm25 = bm25_weight(valid_sparse_user_item, K1=100, B=0.9).tocsr() # Implicit veut des matrices [user x item]

display(train_sparse_user_item_bm25.shape)
display(valid_sparse_user_item_bm25.shape)

(297141, 28002)

(84041, 7576)

### Entrainons un premier modèle pour calculer les embeddings utilisateurs et faire des recommandations

In [17]:
model_bm25 = implicit.als.AlternatingLeastSquares(
    factors=32, 
    regularization=0.05, 
    iterations=50,
    alpha=40
)

model_name = f"{model_bm25.__class__.__name__}_with_BM25"
t0 = time.perf_counter()
model_bm25.fit(train_sparse_user_item)
scores_df.at[model_name,'training_time'] = time.perf_counter() - t0

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [02:28<00:00,  2.96s/it]


### Testons une `recommandation sur la base d'un ou plusieurs utilisateurs`

In [18]:
# Make recommendations for the first 10 users in the dataset
userids = [59, 1024] # liste d'ID utilisateurs
rec_size = 5

codes, scores = model_bm25.recommend(userids, valid_sparse_user_item_bm25[userids], N=rec_size, filter_already_liked_items=True) 

for i, user_id in enumerate(userids):
    print(f"\n --- Liste d'articles candidats pour l'utilisateur {user_id} --- \n")
    
    for code, score in zip(codes[i], scores[i]):
        idx = data_train[data_train.article_cat_code==code]['article_id'].iloc[0]
        print(f"catCode: {code:10} \t article_id: {idx:10} \t score: {score:.2f}")


 --- Liste d'articles candidats pour l'utilisateur 59 --- 

catCode:      22427 	 article_id:     284463 	 score: 0.02
catCode:      12872 	 article_id:     158082 	 score: 0.02
catCode:       8129 	 article_id:      96210 	 score: 0.02
catCode:       3504 	 article_id:      42883 	 score: 0.02
catCode:      17313 	 article_id:     214800 	 score: 0.02

 --- Liste d'articles candidats pour l'utilisateur 1024 --- 

catCode:      26186 	 article_id:     336221 	 score: 0.00
catCode:      26208 	 article_id:     336429 	 score: 0.00
catCode:      13184 	 article_id:     160974 	 score: 0.00
catCode:       9509 	 article_id:     107272 	 score: 0.00
catCode:      16866 	 article_id:     208322 	 score: 0.00


### Testons une `recommandation sur la base d'un article` *(ce n'est pas le but d'un Collaborative Filtering, mais on peut le faire alors autant l'essayer)*

In [19]:
article_id = 162605
article_code = data_train[data_train.article_id == article_id]['article_cat_code'].iloc[0]
rec_size = 5

# Get similar items.
codes, scores = model_bm25.similar_items(article_code, N=rec_size , filter_items=[article_code])

print(f"\n --- Liste d'articles candidats sur la base de l'article {article_id} --- \n")
for code, score in zip(codes, scores):
    idx = data_train[data_train.article_cat_code==code]['article_id'].iloc[0]
    print(f"catCode: {code:10} \t article_id: {idx:10} \t score: {score:.2f}")


 --- Liste d'articles candidats sur la base de l'article 162605 --- 

catCode:       2658 	 article_id:      36530 	 score: 0.73
catCode:      17271 	 article_id:     214455 	 score: 0.72
catCode:      24877 	 article_id:     314431 	 score: 0.71
catCode:      23029 	 article_id:     289096 	 score: 0.71
catCode:       7182 	 article_id:      84246 	 score: 0.71


### Evaluons le modèle

> **Nous devons nous rappeler que la recommandation n'est pas une prédiction.**<br>
> S'appuyer sur des métriques ML pour déterminer la performance d'un système de recommandation n'est pas suffisant.<br>
> Seul **le retour des utilisateurs apporte des résultats valables et c'est pourquoi les tests A/B devraient toujours être priviligiés**.

- Dans la mesure ou notre jeu de données **ne dispose pas de scores explicites**, il ne parrait pas souhaitable d'utiliser des métrique du type `MAE` ou `RMSE`.
- Dans la mesure ou l'on **ne cherche pas particulièrement à obtenir un ordre précis**, il ne parrait pas souhaitable d'utiliser des métriques de ranking comme le `MAP@K` ou le `nDCCG`.
- Nous pourrions donc nous tourner vers la `Precision@k`, le `Recall@K` et donc le `F1@k`, mais il est probable que ce ne soit pas très représentatif.

#### Regardons la precision@k

In [20]:
precision_k = evaluation.precision_at_k(model_bm25, train_sparse_user_item_bm25, valid_sparse_user_item_bm25, K=5, show_progress=True, num_threads=1)
print(precision_k)
scores_df.at[model_name,'precision@k'] = precision_k

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84041/84041 [00:24<00:00, 3413.26it/s]

0.00042019234304502886


> Le probleme c'est même si c'est l'une des métriques disponible les plus adaptée, elle reste peu adaptée à notre problème...<br>
> Ici `Precision = (# of top k recommendations that are relevant)/(# of items that are recommended)`<br>
> Mais malgré un nombre d'article assez large, on ne recommande que 5 articles et les utilisateurs ont un historique assez faible dans notre jeu de données. Donc les chances de recommander un article parmi 5 qui a effectivement été lu ensuite par l'utilisateur sont vraiment faible.

#### Regardons le MAP@k

In [21]:
map_k = evaluation.mean_average_precision_at_k(model_bm25, train_sparse_user_item_bm25, valid_sparse_user_item_bm25, K=5, show_progress=True, num_threads=1)
print(map_k)
scores_df.at[model_name,'map@k'] = map_k

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84041/84041 [00:24<00:00, 3395.50it/s]

0.00015959472162396917


#### Regardons le nDCG@k

In [22]:
ndcg_k = evaluation.ndcg_at_k(model_bm25, train_sparse_user_item_bm25, valid_sparse_user_item_bm25, K=5, show_progress=True, num_threads=1)
print(ndcg_k)
scores_df.at[model_name,'ndcg@k'] = ndcg_k

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84041/84041 [00:24<00:00, 3377.05it/s]

0.0002718383754096874


### Construisons une métrique sur mesure

Pour avoir une idée une métrique utilsable pour comparer nos différents modèles, nous pourrions comparer l'embedding moyen des articles lus APRÈS *(donc les actions contenues dans data_valid)* avec l'embedding moyen des article recommandés ET avec l'embedding moyen des articles recommandés.

#### Calculons la cosine similarity moyenne sur le jeu de validation

In [23]:
def reco_collaborative_filtering_bm25(user_ids, mean_embeddings, reco_ref, reco_size):
    
    apply_numpy2 = lambda x : list(map(lookup_articles,x))
    apply_numpy = lambda x : list(map(apply_numpy2,x))
    
    reco_codes, reco_score = model_bm25.recommend(user_ids, reco_ref[user_ids], N=reco_size, filter_already_liked_items=True) 
    return apply_numpy(reco_codes)

#### Pour tout le jeu de données

In [24]:
MCS, MCS_df = get_mean_cosine_similarity(data_valid, valid_sparse_user_item_bm25, reco_collaborative_filtering_bm25, 5)

display(MCS_df.head(), MCS_df.shape)
print(f"\nmean_cosine_similarity: {MCS:.2f} (sachant que la cosine similarity va de 1 à -1)")

,article_ids,read_mean_embedding,reco_article_ids,reco_mean_embedding,cosine
user_id,,,,,
5,"[225010, 69353, 161872, 205845, 57748, 157815,...","[-0.024590481, -0.9645934, -0.06022447, -0.023...","[87224, 183176, 84136, 62567, 277492]","[-0.18971053, -0.9635401, -0.06036191, -0.2341...",0.656579
7,"[199474, 87223, 352979, 284470, 36162, 156279]","[-0.11400774, -0.96468645, -0.32728586, -0.201...","[160974, 336223, 336499, 123909, 162655]","[-0.29034764, -0.97227734, -0.3853094, -0.4268...",0.737372
8,"[331116, 96141, 234481]","[-0.32962552, -0.97033435, 0.20547153, -0.0870...","[160417, 156624, 315105, 284096, 214800]","[-0.18965398, -0.9585942, -0.12331337, 0.17141...",0.531719
10,"[196588, 193449, 195689, 65991, 100931, 58556,...","[-0.23681411, -0.96241695, -0.08872973, -0.346...","[64409, 272660, 162655, 182394, 336245]","[-0.23325042, -0.97391224, -0.24728794, -0.610...",0.757859
11,"[208582, 96877, 195177, 205824]","[-0.15431535, -0.9709704, 0.069619074, 0.13036...","[336223, 129434, 123757, 234698, 20691]","[-0.21669254, -0.96519125, -0.6087297, -0.4518...",0.439431


(46638, 5)


mean_cosine_similarity: 0.54 (sachant que la cosine similarity va de 1 à -1)


#### Pour les 1000 premiers utilisateurs *(car j'ai du mal à calculer le score sur l'ensemble du jeu de données avec le Content Based Filtering)*

In [25]:
MCS1, MCS1_df = get_mean_cosine_similarity(data_valid, valid_sparse_user_item_bm25, reco_collaborative_filtering_bm25, 5, 1000)
print(f"\nmean_cosine_similarity: {MCS1:.2f} (sachant que la cosine similarity va de 1 à -1)")
scores_df.at[model_name,'mean_cosine_similarity'] = MCS1


mean_cosine_similarity: 0.57 (sachant que la cosine similarity va de 1 à -1)


#### Comparons

In [26]:
scores_df

,mean_cosine_similarity,precision@k,map@k,ndcg@k,training_time
model_name,,,,,
Content Based Filtering - Random,0.482212,NaN,NaN,NaN,0
AlternatingLeastSquares_with_BM25,0.567325,0.00042,0.00016,0.000272,148.21495


## 1.2 Content Based Filtering

articles_embeddings.pickle Pickle (Python 3) of a NumPy matrix containing the Article Content Embeddings (250-dimensional vectors), trained upon articles' text and metadata by the CHAMELEON's ACR module (see paper for details) for 364047 published articles.
P.s. The full text of news articles could not be provided due to license restrictions, but those embeddings can be used by Neural Networks to represent their content. See this paper for a t-SNE visualization of these embeddings, colored by category.

### Trouvons l'embedding moyen des articles lus par un utilisateur donnée

In [27]:
user_id = 20137
history_size = 5

def get_mean_vector(articles_idx):
    apply_numpy = lambda x : article_embedding[x].mean(axis=0)
    return apply_numpy(articles_idx)            

user_articles_idx = data_train[data_train.user_id == user_id]['article_id']
last_articles_idx = user_articles_idx.iloc[-history_size:].values #.sort_values('click_timestamp')
mean_vector = get_mean_vector(last_articles_idx)

print(f"Articles utilisés dans le mean embedding: {last_articles_idx}")

Articles utilisés dans le mean embedding: [288440 337441 202476 250043 284583]


### Calculons la similarité de cet embedding avec les embeddings des articles présents dans note fichier

In [28]:
def get_cosine(article_embedding, mean_vector, user_articles_idx=None):
    
    A = article_embedding.copy()
    B = mean_vector

    # --- Assurons nous de ne pas recommander les article déjà lus par cet utilisateur
    if user_articles_idx is not None:
        A[user_articles_idx] = -B # je donne aux articles lus un embedding inverse de celui que l'on cible
 
    # --- Calculons la cosine similarity entre l'embedding moyen de l'utilisateur et les articles connus
    return np.dot(A,B)/(norm(A, axis=1)*norm(B))

# --- Calculons la cosine similarity entre l'embedding moyen de l'utilisateur et les articles connus
cosine = get_cosine(article_embedding, mean_vector, user_articles_idx)
print("Cosine Similarity:", cosine, cosine.shape)

Cosine Similarity: [0.32582363 0.2783536  0.31348327 ... 0.38568467 0.2742523  0.42668363] (364047,)


### Recommandons 5 articles à l'utilisateur

In [29]:
def recommend_articles(cosine, pred_size=5):
    cos = pd.DataFrame(cosine, columns=['cosine_sim'])
    selection = cos.sort_values('cosine_sim', ascending=False)[:pred_size]
    selection.reset_index(inplace=True)
    selection.rename(columns={'index':'article_id'}, inplace=True)
    return selection

reco = recommend_articles(cosine, 5)
reco

,article_id,cosine_sim
0,284768,0.830968
1,345593,0.821919
2,285424,0.821205
3,345566,0.818335
4,283576,0.816523


### Comparons avec les articles consultés par cet utilisateur dans le validation_set

In [30]:
mean_vector_recommended = get_mean_vector(reco.article_id)

In [31]:
viewed = data_valid[data_valid.user_id == user_id]['article_id'].values
mean_vector_viewed = get_mean_vector(viewed)

#### Similarité entre les articles lus dans le `valid_set` et les articles prédis

In [32]:
def get_cosine_similarity(A, B):
    return np.dot(A,B)/(norm(A)*norm(B))

cosine_similarity_score = get_cosine_similarity(mean_vector_recommended, mean_vector_viewed)
print("Cosine Similarity:", cosine_similarity_score)

Cosine Similarity: 0.71754134


#### Similarité entre les articles lus dans le `train_set` et les articles lus dans le `valid_set`

In [33]:
cosine_similarity_score = get_cosine_similarity(mean_vector, mean_vector_viewed)
print("Cosine Similarity:", cosine_similarity_score)

Cosine Similarity: 0.7984939


#### Calculons la cosine similarity moyenne sur l'ensemble du jeu de validation *(⚠️ en fait une fraction car c'est beaucoup trop lent)*

In [34]:
cosine_pred_viewed = []
reco_size = 5

# --- get users of the validation set
user_idx = np.sort(data_valid.user_id.unique())

for user_id in user_idx[:1000]:
    #print(user_id)
    
    # --- Compute viewed mean_embedding
    viewed_train = data_train[data_train.user_id == user_id]['article_id']
    if len(viewed_train) < 1: continue
    mean_vector_viewed_train = get_mean_vector(viewed_train.values)
    
    # --- Compute viewed mean_embedding 
    viewed_valid = data_valid[data_valid.user_id == user_id]['article_id']
    if len(viewed_valid) < 1: continue
    mean_vector_viewed_valid = get_mean_vector(viewed_valid.values)
    
    # --- Compute cosine similarity the mean of the viewed articles and the rest
    A = article_embedding.copy()
    B = mean_vector_viewed_train
    A[viewed_train.values] = -B # Cancel already read articles
    cosine = cosine_similarity(A, B.reshape(1, -1))
    
    recommendations =  recommend_articles(cosine, reco_size)
    mean_vector_reco = get_mean_vector(recommendations.article_id)
    
    # --- Compute similarities
    cosine_similarity_pred_viewed = cosine_similarity(mean_vector_reco.reshape(1, -1), mean_vector_viewed_valid.reshape(1, -1))
    
    cosine_pred_viewed.append(cosine_similarity_pred_viewed)

In [35]:
print(f"\nmean_cosine_similarity (de 1 à -1) entre 'recommended' & 'viewed_in_valid' = {np.mean(cosine_pred_viewed):.2f} (que l'on compare donc avec les 0.55 du Collaborative Filtering ALS)")

model_name = "Content Based Filtering"
scores_df.at[model_name,'mean_cosine_similarity'] = np.mean(cosine_pred_viewed)
scores_df.at[model_name,'training_time'] = 0


mean_cosine_similarity (de 1 à -1) entre 'recommended' & 'viewed_in_valid' = 0.58 (que l'on compare donc avec les 0.55 du Collaborative Filtering ALS)


#### Comparons

In [36]:
scores_df

,mean_cosine_similarity,precision@k,map@k,ndcg@k,training_time
model_name,,,,,
Content Based Filtering - Random,0.482212,NaN,NaN,NaN,0
AlternatingLeastSquares_with_BM25,0.567325,0.00042,0.00016,0.000272,148.21495
Content Based Filtering,0.576889,NaN,NaN,NaN,0


> #### Si l'on se fie au score moyen de similarité entre les recommandations et ce qui a été effectivemet lu (dans le validation set), le `Collaborative filtering` semble plus précis.
> Mais ce n'est pas le seul élément à prendre en compte *(d'autant que cette mesure n'est pas très significative)*.

> `Collaborative Filtering`:
> * plus en accord avec ce qui est dans le validation set par les utilisateurs,
> * plus rapide,
> * limité aux articles visités.<br>

> `Content Based Filtering`:
> * plus lent *(du moins je n'ai pas réussi à le rendre rapide)*,
> * moins prévis *(mais je n'ai pu évaluer que les 100 premiers utlisateurs)*
> * peut recommander n'importe quel article dont on a l'embedding, y compris ceux jamais visités par un utilisateur.

> Dans les deux cas, les modèles employés ne permettent pas de prendre en compte des features supplémentaires comme par exemple la catégorie de l'article etc. *(Il faudrait un Collaborative Filtering en DNN)*

> L'idéal est donc probablement d'utiliser une combinaison des deux approches; le Collaboravie filtering pour être le plus proche possible des intérêts de l'utilisateur, et le Content Based Filtering pour apporter de la diversité sans pour autant trop s'éloigner des sujets de prédilection de l'utilisateur.

## 1.3 Essayons quelques variantes

### AlternatingLeastSquares sans BM25

#### Entrainons un premier modèle pour calculer les embeddings utilisateurs et faire des recommandations

In [37]:
model_als = implicit.als.AlternatingLeastSquares(
    factors=32, 
    regularization=0.05, 
    iterations=50,
    alpha=40
)

model_name = model_als.__class__.__name__
t0 = time.perf_counter()
model_als.fit(train_sparse_user_item)
scores_df.at[model_name,'training_time'] = time.perf_counter() - t0

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [02:23<00:00,  2.88s/it]


#### Calculons la cosine similarity moyenne sur le jeu de validation

In [38]:
def reco_collaborative_filtering_als(user_ids, mean_embeddings, reco_ref, reco_size):
    
    apply_numpy2 = lambda x : list(map(lookup_articles,x))
    apply_numpy = lambda x : list(map(apply_numpy2,x))
    
    reco_codes, reco_score = model_als.recommend(user_ids, reco_ref[user_ids], N=reco_size, filter_already_liked_items=True) 
    return apply_numpy(reco_codes)

Pour tout le jeu de données

In [39]:
MCS1als, MCS1als_df = get_mean_cosine_similarity(data_valid, valid_sparse_user_item, reco_collaborative_filtering_als, 5)
print(f"\nmean_cosine_similarity: {MCS1als:.2f} (sachant que la cosine similarity va de 1 à -1)")


mean_cosine_similarity: 0.52 (sachant que la cosine similarity va de 1 à -1)


Pour les 1000 premiers utilisateurs *(car j'ai du mal à calculer le score sur l'ensemble du jeu de données avec le Content Based Filtering)*

In [40]:
MCS1als, MCS1als_df = get_mean_cosine_similarity(data_valid, valid_sparse_user_item, reco_collaborative_filtering_als, 5, 1000)
print(f"\nmean_cosine_similarity: {MCS1als:.2f} (sachant que la cosine similarity va de 1 à -1)")
scores_df.at[model_name,'mean_cosine_similarity'] = MCS1als


mean_cosine_similarity: 0.56 (sachant que la cosine similarity va de 1 à -1)


#### Regardons la precision@k

In [41]:
precision_k = evaluation.precision_at_k(model_als, train_sparse_user_item, valid_sparse_user_item, K=5, show_progress=True, num_threads=1)
print(precision_k)
scores_df.at[model_name,'precision@k'] = precision_k

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84041/84041 [00:23<00:00, 3507.50it/s]

0.0003361538744360231


#### Regardons le MAP@k

In [42]:
map_k = evaluation.mean_average_precision_at_k(model_als, train_sparse_user_item, valid_sparse_user_item, K=5, show_progress=True, num_threads=1)
print(map_k)
scores_df.at[model_name,'map@k'] = map_k

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84041/84041 [00:25<00:00, 3348.88it/s]

0.00011138743126701382


#### Regardons le nDCG@k

In [43]:
ndcg_k = evaluation.ndcg_at_k(model_als, train_sparse_user_item, valid_sparse_user_item, K=5, show_progress=True, num_threads=1)
print(ndcg_k)
scores_df.at[model_name,'ndcg@k'] = ndcg_k

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84041/84041 [00:25<00:00, 3334.59it/s]

0.0002021168675401838


#### Comparons

In [44]:
scores_df

,mean_cosine_similarity,precision@k,map@k,ndcg@k,training_time
model_name,,,,,
Content Based Filtering - Random,0.482212,NaN,NaN,NaN,0
AlternatingLeastSquares_with_BM25,0.567325,0.00042,0.00016,0.000272,148.21495
Content Based Filtering,0.576889,NaN,NaN,NaN,0
AlternatingLeastSquares,0.560736,0.000336,0.000111,0.000202,143.944907


### Logistic Matrix Factorization

#### Entrainons un premier modèle pour calculer les embeddings utilisateurs et faire des recommandations

In [45]:
model_lmf = implicit.cpu.lmf.LogisticMatrixFactorization(
    factors=32,
    learning_rate=0.05,
    regularization=0.05, 
    iterations=50,
)

model_name = model_lmf.__class__.__name__
t0 = time.perf_counter()
model_lmf.fit(train_sparse_user_item)
scores_df.at[model_name,'training_time'] = time.perf_counter() - t0

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:25<00:00,  1.97it/s]


#### Calculons la cosine similarity moyenne sur le jeu de validation

In [46]:
def reco_collaborative_filtering_lmf(user_ids, mean_embeddings, reco_ref, reco_size):
    
    apply_numpy2 = lambda x : list(map(lookup_articles,x))
    apply_numpy = lambda x : list(map(apply_numpy2,x))
    
    reco_codes, reco_score = model_lmf.recommend(user_ids, reco_ref[user_ids], N=reco_size, filter_already_liked_items=True) 
    return apply_numpy(reco_codes)

Pour tout le jeu de données

In [47]:
MCS1lmf, MCS1lmf_df = get_mean_cosine_similarity(data_valid, valid_sparse_user_item, reco_collaborative_filtering_lmf, 5)
print(f"\nmean_cosine_similarity: {MCS1lmf:.2f} (sachant que la cosine similarity va de 1 à -1)")


mean_cosine_similarity: 0.53 (sachant que la cosine similarity va de 1 à -1)


Pour les 1000 premiers utilisateurs *(car j'ai du mal à calculer le score sur l'ensemble du jeu de données avec le Content Based Filtering)*

In [48]:
MCS1lmf, MCS1lmf_df = get_mean_cosine_similarity(data_valid, valid_sparse_user_item, reco_collaborative_filtering_lmf, 5, 1000)
print(f"\nmean_cosine_similarity: {MCS1lmf:.2f} (sachant que la cosine similarity va de 1 à -1)")
scores_df.at[model_name,'mean_cosine_similarity'] = MCS1lmf


mean_cosine_similarity: 0.56 (sachant que la cosine similarity va de 1 à -1)


#### Regardons la precision@k

In [49]:
precision_k = evaluation.precision_at_k(model_lmf, train_sparse_user_item, valid_sparse_user_item, K=5, show_progress=True, num_threads=1)
print(precision_k)
scores_df.at[model_name,'precision@k'] = precision_k

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84041/84041 [00:24<00:00, 3462.48it/s]

0.0001470673200657601


#### Regardons le MAP@k

In [50]:
map_k = evaluation.mean_average_precision_at_k(model_lmf, train_sparse_user_item, valid_sparse_user_item, K=5, show_progress=True, num_threads=1)
print(map_k)
scores_df.at[model_name,'map@k'] = map_k

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84041/84041 [00:24<00:00, 3399.66it/s]

6.107468709056029e-05


#### Regardons le nDCG@k

In [51]:
ndcg_k = evaluation.ndcg_at_k(model_lmf, train_sparse_user_item, valid_sparse_user_item, K=5, show_progress=True, num_threads=1)
print(ndcg_k)
scores_df.at[model_name,'ndcg@k'] = ndcg_k

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84041/84041 [00:24<00:00, 3377.17it/s]

0.00010342046358378744


#### Comparons

In [52]:
scores_df

,mean_cosine_similarity,precision@k,map@k,ndcg@k,training_time
model_name,,,,,
Content Based Filtering - Random,0.482212,NaN,NaN,NaN,0
AlternatingLeastSquares_with_BM25,0.567325,0.00042,0.00016,0.000272,148.21495
Content Based Filtering,0.576889,NaN,NaN,NaN,0
AlternatingLeastSquares,0.560736,0.000336,0.000111,0.000202,143.944907
LogisticMatrixFactorization,0.558146,0.000147,0.000061,0.000103,25.788472


### Bayesian Personalized Ranking

#### Entrainons un premier modèle pour calculer les embeddings utilisateurs et faire des recommandations

In [53]:
model_bpr = implicit.cpu.bpr.BayesianPersonalizedRanking(
    factors=32,
    learning_rate=0.05,
    regularization=0.05, 
    iterations=50,
)

model_name = model_bpr.__class__.__name__
t0 = time.perf_counter()
model_bpr.fit(train_sparse_user_item)
scores_df.at[model_name,'training_time'] = time.perf_counter() - t0

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:08<00:00,  6.21it/s, train_auc=88.20%, skipped=4.24%]


#### Calculons la cosine similarity moyenne sur le jeu de validation

In [54]:
def reco_collaborative_filtering_bpr(user_ids, mean_embeddings, reco_ref, reco_size):
    
    apply_numpy2 = lambda x : list(map(lookup_articles,x))
    apply_numpy = lambda x : list(map(apply_numpy2,x))
    
    reco_codes, reco_score = model_bpr.recommend(user_ids, reco_ref[user_ids], N=reco_size, filter_already_liked_items=True) 
    return apply_numpy(reco_codes)

Pour tout le jeu de données

In [55]:
MCS1bpr, MCS1bpr_df = get_mean_cosine_similarity(data_valid, valid_sparse_user_item, reco_collaborative_filtering_bpr, 5)
print(f"\nmean_cosine_similarity: {MCS1bpr:.2f} (sachant que la cosine similarity va de 1 à -1)")


mean_cosine_similarity: 0.49 (sachant que la cosine similarity va de 1 à -1)


Pour les 1000 premiers utilisateurs *(car j'ai du mal à calculer le score sur l'ensemble du jeu de données avec le Content Based Filtering)*

In [56]:
MCS1bpr, MCS1bpr_df = get_mean_cosine_similarity(data_valid, valid_sparse_user_item, reco_collaborative_filtering_bpr, 5, 1000)
print(f"\nmean_cosine_similarity: {MCS1bpr:.2f} (sachant que la cosine similarity va de 1 à -1)")
scores_df.at[model_name,'mean_cosine_similarity'] = MCS1bpr


mean_cosine_similarity: 0.52 (sachant que la cosine similarity va de 1 à -1)


#### Regardons la precision@k

In [57]:
precision_k = evaluation.precision_at_k(model_bpr, train_sparse_user_item, valid_sparse_user_item, K=5, show_progress=True, num_threads=1)
print(precision_k)
scores_df.at[model_name,'precision@k'] = precision_k

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84041/84041 [00:24<00:00, 3478.36it/s]

0.00019433895865832585


#### Regardons le MAP@k

In [58]:
map_k = evaluation.mean_average_precision_at_k(model_bpr, train_sparse_user_item, valid_sparse_user_item, K=5, show_progress=True, num_threads=1)
print(map_k)
scores_df.at[model_name,'map@k'] = map_k

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84041/84041 [00:24<00:00, 3376.74it/s]

7.729692782226663e-05


#### Regardons le nDCG@k

In [59]:
ndcg_k = evaluation.ndcg_at_k(model_bpr, train_sparse_user_item, valid_sparse_user_item, K=5, show_progress=True, num_threads=1)
print(ndcg_k)
scores_df.at[model_name,'ndcg@k'] = ndcg_k

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84041/84041 [00:24<00:00, 3434.38it/s]

0.00012868482268411547


#### Comparons

In [60]:
scores_df

,mean_cosine_similarity,precision@k,map@k,ndcg@k,training_time
model_name,,,,,
Content Based Filtering - Random,0.482212,NaN,NaN,NaN,0
AlternatingLeastSquares_with_BM25,0.567325,0.00042,0.00016,0.000272,148.21495
Content Based Filtering,0.576889,NaN,NaN,NaN,0
AlternatingLeastSquares,0.560736,0.000336,0.000111,0.000202,143.944907
LogisticMatrixFactorization,0.558146,0.000147,0.000061,0.000103,25.788472
BayesianPersonalizedRanking,0.521805,0.000194,0.000077,0.000129,8.248168


### Item Item Recommender

In [61]:
model_name = "ItemItemRecommender"

#### Entrainons un premier modèle pour calculer les embeddings utilisateurs et faire des recommandations

In [62]:
model_iir = implicit.nearest_neighbours.ItemItemRecommender()

model_name = model_iir.__class__.__name__
t0 = time.perf_counter()
model_iir.fit(train_sparse_user_item)
scores_df.at[model_name,'training_time'] = time.perf_counter() - t0

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28002/28002 [00:00<00:00, 77735.12it/s]


#### Calculons la cosine similarity moyenne sur le jeu de validation

In [63]:
def reco_collaborative_filtering_iir(user_ids, mean_embeddings, reco_ref, reco_size):
    
    apply_numpy2 = lambda x : list(map(lookup_articles,x))
    apply_numpy = lambda x : list(map(apply_numpy2,x))
    
    reco_codes, reco_score = model_iir.recommend(user_ids, reco_ref[user_ids], N=reco_size, filter_already_liked_items=True) 
    return apply_numpy(reco_codes)

Pour tout le jeu de données

In [64]:
MCS1iir, MCS1iir_df = get_mean_cosine_similarity(data_valid, valid_sparse_user_item, reco_collaborative_filtering_iir, 5)
print(f"\nmean_cosine_similarity: {MCS1iir:.2f} (sachant que la cosine similarity va de 1 à -1)")


mean_cosine_similarity: 0.47 (sachant que la cosine similarity va de 1 à -1)


Pour les 1000 premiers utilisateurs *(car j'ai du mal à calculer le score sur l'ensemble du jeu de données avec le Content Based Filtering)*

In [65]:
MCS1iir, MCS1iir_df = get_mean_cosine_similarity(data_valid, valid_sparse_user_item, reco_collaborative_filtering_iir, 5, 1000)
print(f"\nmean_cosine_similarity: {MCS1iir:.2f} (sachant que la cosine similarity va de 1 à -1)")
scores_df.at[model_name,'mean_cosine_similarity'] = MCS1iir


mean_cosine_similarity: 0.51 (sachant que la cosine similarity va de 1 à -1)


#### Regardons la precision@k

In [66]:
precision_k = evaluation.precision_at_k(model_iir, train_sparse_user_item, valid_sparse_user_item, K=5, show_progress=True, num_threads=1)
print(precision_k)
scores_df.at[model_name,'precision@k'] = precision_k

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84041/84041 [00:09<00:00, 9120.22it/s]

0.00042019234304502886


#### Regardons le MAP@k

In [67]:
map_k = evaluation.mean_average_precision_at_k(model_iir, train_sparse_user_item, valid_sparse_user_item, K=5, show_progress=True, num_threads=1)
print(map_k)
scores_df.at[model_name,'map@k'] = map_k

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84041/84041 [00:09<00:00, 9259.70it/s]

0.00019785316426241682


#### Regardons le nDCG@k

In [68]:
ndcg_k = evaluation.ndcg_at_k(model_iir, train_sparse_user_item, valid_sparse_user_item, K=5, show_progress=True, num_threads=1)
print(ndcg_k)
scores_df.at[model_name,'ndcg@k'] = ndcg_k

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84041/84041 [00:09<00:00, 9187.37it/s]

0.0003075531886240062


#### Comparons

In [69]:
scores_df

,mean_cosine_similarity,precision@k,map@k,ndcg@k,training_time
model_name,,,,,
Content Based Filtering - Random,0.482212,NaN,NaN,NaN,0
AlternatingLeastSquares_with_BM25,0.567325,0.00042,0.00016,0.000272,148.21495
Content Based Filtering,0.576889,NaN,NaN,NaN,0
AlternatingLeastSquares,0.560736,0.000336,0.000111,0.000202,143.944907
LogisticMatrixFactorization,0.558146,0.000147,0.000061,0.000103,25.788472
BayesianPersonalizedRanking,0.521805,0.000194,0.000077,0.000129,8.248168
ItemItemRecommender,0.505485,0.00042,0.000198,0.000308,0.441054


### Content Based Filtering basé sur le dernier article lu uniquement

In [70]:
cosine_pred_viewed = []
reco_size = 5

# --- get users of the validation set
user_idx = np.sort(data_valid.user_id.unique())

for user_id in user_idx[:1000]:
    #print(user_id)
    
    # --- Compute viewed mean_embedding
    #viewed_train = data_train[data_train.user_id == user_id]['article_id'].iloc[:1]
    #if len(viewed_train) < 1: continue
    #mean_vector_viewed_train = get_mean_vector(viewed_train.values)
    
    viewed = data_train[data_train.user_id == user_id]['article_id']
    if len(viewed) < 1: continue
    mean_vector_viewed = article_embedding[max(viewed)]
    
    # --- Compute viewed mean_embedding 
    #viewed_valid = data_valid[data_valid.user_id == user_id]['article_id'].iloc[:1]
    #if len(viewed_valid) < 1: continue
    #mean_vector_viewed_valid = get_mean_vector(viewed_valid.values)
    
    viewed = data_valid[data_valid.user_id == user_id]['article_id']
    if len(viewed) < 1: continue
    mean_vector_viewed = article_embedding[max(viewed)]
    
    # --- Compute cosine similarity the mean of the viewed articles and the rest
    A = article_embedding.copy()
    B = mean_vector_viewed_train
    A[viewed_train.values] = -B # Cancel already read articles
    cosine = cosine_similarity(A, B.reshape(1, -1))
    
    recommendations =  recommend_articles(cosine, reco_size)
    mean_vector_reco = get_mean_vector(recommendations.article_id)
    
    # --- Compute similarities
    cosine_similarity_pred_viewed = cosine_similarity(mean_vector_reco.reshape(1, -1), mean_vector_viewed_valid.reshape(1, -1))
    
    cosine_pred_viewed.append(cosine_similarity_pred_viewed)

In [71]:
print(f"\nmean_cosine_similarity (de 1 à -1) entre 'recommended' & 'viewed_in_valid' = {np.mean(cosine_pred_viewed):.2f} (que l'on compare donc avec les 0.55 du Collaborative Filtering ALS)")

model_name = "Content Based Filtering based on last read article"
scores_df.at[model_name,'mean_cosine_similarity'] = np.mean(cosine_pred_viewed)
scores_df.at[model_name,'training_time'] = 0


mean_cosine_similarity (de 1 à -1) entre 'recommended' & 'viewed_in_valid' = 0.76 (que l'on compare donc avec les 0.55 du Collaborative Filtering ALS)


## 1.3 Revue des scores <a class="anchor" id="models_scores"></a> [⇪](#menu)

In [72]:
scores_df

,mean_cosine_similarity,precision@k,map@k,ndcg@k,training_time
model_name,,,,,
Content Based Filtering - Random,0.482212,NaN,NaN,NaN,0
AlternatingLeastSquares_with_BM25,0.567325,0.00042,0.00016,0.000272,148.21495
Content Based Filtering,0.576889,NaN,NaN,NaN,0
AlternatingLeastSquares,0.560736,0.000336,0.000111,0.000202,143.944907
LogisticMatrixFactorization,0.558146,0.000147,0.000061,0.000103,25.788472
BayesianPersonalizedRanking,0.521805,0.000194,0.000077,0.000129,8.248168
ItemItemRecommender,0.505485,0.00042,0.000198,0.000308,0.441054
Content Based Filtering based on last read article,0.762165,NaN,NaN,NaN,0
